<a href="https://colab.research.google.com/github/sanvidhaharibhakta/Practicals/blob/main/Matrix_Multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/afnan47/cuda.git


  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-q242oj3k
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-q242oj3k
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=493792cd9e536dc9c05ce830abb7b70eba1d17c06590a44982345255dfe3fcc5
  Stored in directory: /tmp/pip-ephem-wheel-cache-t44b96v_/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
%%writefile matrix_mult.cu
#include <iostream>
#include <cuda.h>
using namespace std;
#define BLOCK_SIZE 1
__global__ void gpuMM(float *A, float *B, float *C, int N) {
int row = blockIdx.y * blockDim.y + threadIdx.y;
int col = blockIdx.x * blockDim.x + threadIdx.x;
float sum = 0.f;
for (int n = 0; n < N; ++n)
sum += A[row * N + n] * B[n * N + col];
C[row * N + col] = sum;
}
int main(int argc, char *argv[]) {
int N;
// Get matrix size from user
cout << "Enter size of matrix (N): ";
cin >> N;
if (N % BLOCK_SIZE != 0) {
cerr << "Matrix size must be a multiple of BLOCK_SIZE." << endl;
return 1;
}
cout << "\nExecuting Matrix Multiplication" << endl;
cout << "Matrix size: " << N << "x" << N << endl;
// Allocate memory for matrices on the host
float *hA, *hB, *hC;
hA = new float[N * N];
hB = new float[N * N];
hC = new float[N * N];
// Read matrices from user
cout << "Enter elements of matrix A (" << N << "x" << N << "):" << endl;
for (int i = 0; i < N * N; ++i)
cin >> hA[i];
cout << "Enter elements of matrix B (" << N << "x" << N << "):" << endl;
for (int i = 0; i < N * N; ++i)
cin >> hB[i];
// Allocate memory for matrices on the device
int size = N * N * sizeof(float);
float *dA, *dB, *dC;
cudaMalloc(&dA, size);
cudaMalloc(&dB, size);
cudaMalloc(&dC, size);
// Copy matrices from the host to the device
cudaMemcpy(dA, hA, size, cudaMemcpyHostToDevice);
cudaMemcpy(dB, hB, size, cudaMemcpyHostToDevice);
dim3 threadBlock(BLOCK_SIZE, BLOCK_SIZE);
dim3 grid(N / BLOCK_SIZE, N / BLOCK_SIZE);
// Execute the matrix multiplication kernel
gpuMM<<<grid, threadBlock>>>(dA, dB, dC, N);
// Copy the result matrix from the device to the host
cudaMemcpy(hC, dC, size, cudaMemcpyDeviceToHost);
// Display the result matrix
cout << "\nResultant matrix:\n";
for (int row = 0; row < N; row++) {
for (int col = 0; col < N; col++) {
cout << hC[row * N + col] << " ";
}
cout << endl;
}
// Free device memory
cudaFree(dA);
cudaFree(dB);
cudaFree(dC);
// Free host memory
delete[] hA;
delete[] hB;
delete[] hC;
cout << "Finished." << endl;
return 0;
}

Writing matrix_mult.cu


In [5]:
!nvcc matrix_mult.cu -o matrix_mult

In [8]:
!./matrix_mult

Enter size of matrix (N): 3

Executing Matrix Multiplication
Matrix size: 3x3
Enter elements of matrix A (3x3):
1 2 3
1 2 3
1 2 3
Enter elements of matrix B (3x3):
2
3 4
2 3 4
2 3 4

Resultant matrix:
12 18 24 
12 18 24 
12 18 24 
Finished.
